# Learn to calculate with seq2seq model

In this assignment, we will learn how to use neural networks to solve sequence-to-sequence prediction tasks. Seq2Seq models are very popular these days because they achieve great results in Machine Translation, Text Summarization, Conversational Modeling and more.

Using sequence-to-sequence modeling, we are going to build a calculator for evaluating arithmetic expressions for addition and subtraction, by taking an equation as an input to the neural network and producing an answer as it's output.

The resulting solution for this problem will be based on state-of-the-art approaches for sequence-to-sequence learning and we could easily adapt it to solve other tasks. However, for training a machine translation system or intellectual chat bot, it would be useful to have access to compute resources like GPU as training such systems is usually time consuming. 


### Data

For this task, we don't need to download any data — we will generate it on our own! We will use two operators (addition and subtraction) and work with positive integer numbers in some range. Here are examples of correct inputs and outputs:

    Input: '1+2'
    Output: '3'
    
    Input: '0-99'
    Output: '-99'

*Note, that there are no spaces between operators and operands.*


Now we need to implement the function *generate_equations*, which will be used to generate the data.

In [1]:
import random
import numpy as np
import itertools
from sklearn.metrics import mean_absolute_error

In [2]:
allowed_operators = ['+', '-']

5

## Generating Data

In [4]:
def generate_equations(allowed_operators, dataset_size, min_value, max_value):
    """Generates pairs of equations and solutions to them.
    
       Each equation has a form of two integers with an operator in between.
       Each solution is an integer with the result of the operaion.
    
        allowed_operators: list of strings, allowed operators.
        dataset_size: an integer, number of equations to be generated.
        min_value: an integer, min value of each operand.
        max_value: an integer, max value of each operand.

        result: a list of tuples of strings (equation, solution).
    """
    sample = []
    
    rand = [(random.randint(min_value, max_value),random.randint(min_value, max_value)) for _ in range(dataset_size)]
    equations = ['{}{}{}'.format(rand[i][0],random.choice(allowed_operators),rand[i][1]) for i in range(dataset_size)]
    solutions = [str(eval(equation)) for equation in equations]
    for equation, solution in zip(equations,solutions):
        sample.append((equation,solution))
        
        
    return sample

To check the correctness of your implementation, use *test_generate_equations* function:

In [5]:
def test_generate_equations():
    allowed_operators = ['+', '-']
    dataset_size = 10
    for (input_, output_) in generate_equations(allowed_operators, dataset_size, 0, 100):
        if not (type(input_) is str and type(output_) is str):
            return "Both parts should be strings."
        if eval(input_) != int(output_):
            return "The (equation: {!r}, solution: {!r}) pair is incorrect.".format(input_, output_)
    return "Tests passed."

In [6]:
print(test_generate_equations())

Tests passed.


Finally, we are ready to generate the train and test data for the neural network:

In [8]:
allowed_operators = ['+', '-']
dataset_size = 100000
data = generate_equations(allowed_operators, dataset_size, min_value=0, max_value=9999)

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

## Prepare data for the neural network

The next stage of data preparation is creating mappings of the characters to their indices in some vocabulary. Since in our task we already know which symbols will appear in the inputs and outputs, generating the vocabulary is a simple step.

#### How to create dictionaries for other task

First of all, we need to understand what is the basic unit of the sequence in the task. In our case, we operate on symbols and the basic unit is a symbol. The number of symbols is small, so we don't need to think about filtering/normalization steps. However, in other tasks, the basic unit is often a word, and in this case the mapping would be *word $\to$ integer*. The number of words might be huge, so it would be reasonable to filter them, for example, by frequency and leave only the frequent ones. Other strategies that your should consider are: data normalization (lowercasing, tokenization, how to consider punctuation marks), separate vocabulary for input and for output (e.g. for machine translation), some specifics of the task.

In [9]:
word2id = {symbol:i for i, symbol in enumerate('#^$+-1234567890')}
id2word = {i:symbol for symbol, i in word2id.items()}

#### Special symbols

In [10]:
start_symbol = '^'
end_symbol = '$'
padding_symbol = '#'

You could notice that we have added 3 special symbols: '^', '\$' and '#':
- '^' symbol will be passed to the network to indicate the beginning of the decoding procedure. We will discuss this one later in more details.
- '\$' symbol will be used to indicate the *end of a string*, both for input and output sequences. 
- '#' symbol will be used as a *padding* character to make lengths of all strings equal within one training batch.


#### Padding

When vocabularies are ready, we need to be able to convert a sentence to a list of vocabulary word indices and back. At the same time, let's care about padding. We are going to preprocess each sequence from the input (and output ground truth) in such a way that:
- it has a predefined length *padded_len*
- it is probably cut off or padded with the *padding symbol* '#'
- it *always* ends with the *end symbol* '$'

We will treat the original characters of the sequence **and the end symbol** as the valid part of the input. We will store *the actual length* of the sequence, which includes the end symbol, but does not include the padding symbols. 

 Now we need to implement the function *sentence_to_ids* that does the described job. 

In [12]:
def sentence_to_ids(sentence, word2id, padded_len):
    """ Converts a sequence of symbols to a padded sequence of their ids.
    
      sentence: a string, input/output sequence of symbols.
      word2id: a dict, a mapping from original symbols to ids.
      padded_len: an integer, a desirable length of the sequence.

      result: a tuple of (a list of ids, an actual length of sentence).
    """
    
    if padded_len <= len(sentence):
        sentence = sentence[:padded_len-1] + '$'
        sen_len = padded_len
    else: 
        sen_len = len(sentence)+1 
        sentence = sentence + '$' + (padded_len-len(sentence)-1)*'#'


    sen_ids = [word2id[x] for x in sentence]
    
    return sen_ids, sen_len

Check that the implementation is correct:

In [13]:
def test_sentence_to_ids():
    sentences = [("123+123", 7), ("123+123", 8), ("123+123", 10)]
    expected_output = [([5, 6, 7, 3, 5, 6, 2], 7), 
                       ([5, 6, 7, 3, 5, 6, 7, 2], 8), 
                       ([5, 6, 7, 3, 5, 6, 7, 2, 0, 0], 8)] 
    for (sentence, padded_len), (sentence_ids, expected_length) in zip(sentences, expected_output):
        output, length = sentence_to_ids(sentence, word2id, padded_len)
        if output != sentence_ids:
            return("Convertion of '{}' for padded_len={} to {} is incorrect.".format(
                sentence, padded_len, output))
        if length != expected_length:
            return("Convertion of '{}' for padded_len={} has incorrect actual length {}.".format(
                sentence, padded_len, length))
    return("Tests passed.")

In [14]:
print(test_sentence_to_ids())

Tests passed.


We also need to be able to get back from indices to symbols:

In [15]:
def ids_to_sentence(ids, id2word):
    """ Converts a sequence of ids to a sequence of symbols.
    
          ids: a list, indices for the padded sequence.
          id2word:  a dict, a mapping from ids to original symbols.

          result: a list of symbols.
    """
 
    return [id2word[i] for i in ids] 

#### Generating batches

The final step of data preparation is a function that transforms a batch of sentences to a list of lists of indices. 

In [16]:
def batch_to_ids(sentences, word2id, max_len):
    """Prepares batches of indices. 
    
       Sequences are padded to match the longest sequence in the batch,
       if it's longer than max_len, then max_len is used instead.

        sentences: a list (or tuple) of strings, original sequences.
        word2id: a dict, a mapping from original symbols to ids.
        max_len: an integer, max len of sequences allowed.

        result: a list of lists of ids, a list of actual lengths.
    """
    
    max_len_in_batch = min(max(len(s) for s in sentences) + 1, max_len)
    batch_ids, batch_ids_len = [], []
    for sentence in sentences:
        ids, ids_len = sentence_to_ids(sentence, word2id, max_len_in_batch)
        batch_ids.append(ids)
        batch_ids_len.append(ids_len)
    return batch_ids, batch_ids_len

The function *generate_batches* will help to generate batches with defined size from given samples.

In [17]:
def generate_batches(samples, batch_size=64):
    X, Y = [], []
    for i, (x, y) in enumerate(samples, 1):
        X.append(x)
        Y.append(y)
        if i % batch_size == 0:
            yield X, Y
            X, Y = [], []
    if X and Y:
        yield X, Y

To illustrate the result of the implemented functions, run the following cell:

In [18]:
sentences = train_set[0]
ids, sent_lens = batch_to_ids(sentences, word2id, max_len=10)
print('Input:', sentences)
print('Ids: {}\nSentences lengths: {}'.format(ids, sent_lens))

Input: ('2743+1676', '4419')
Ids: [[6, 11, 8, 7, 3, 5, 10, 11, 10, 2], [8, 8, 5, 13, 2, 0, 0, 0, 0, 0]]
Sentences lengths: [10, 5]


This is to see what generate_batches does.

## Encoder-Decoder architecture

Encoder-Decoder is a successful architecture for Seq2Seq tasks with different lengths of input and output sequences. The main idea is to use two recurrent neural networks, where the first neural network *encodes* the input sequence into a real-valued vector and then the second neural network *decodes* this vector into the output sequence. While building the neural network, we will specify some particular characteristics of this architecture.

Let us use TensorFlow building blocks to specify the network architecture.

Now, let us specify the layers of the neural network. First, we need to prepare an embedding layer, which can be used for both  input and output. A random matrix followed by a lookup function will do as well. But embedding layers automatically pass the mask to the mask-consuming layers. So we dont use input lengths. In general, for tasks with different vocabularies there would be multiple embedding layers.

#### Encoder

The first RNN of the current architecture is called an *encoder* and serves for encoding an input sequence to a real-valued vector. Input of this RNN is an embedded input batch. Since sentences in the same batch could have different actual lengths, it is useful to provide input lengths to avoid unnecessary computations. This is done by masking produced by embedding layers. The final encoder state will be passed to the second RNN (decoder), which we will create soon. 

#### Decoder

The second RNN is called a *decoder* and serves for generating the output sequence. In the simple seq2seq arcitecture, the input sequence is provided to the decoder only as the final state of the encoder. Obviously, it is a bottleneck and [Attention techniques](https://www.tensorflow.org/tutorials/seq2seq#background_on_the_attention_mechanism) can help to overcome it, however we do not need them to make our calculator work though this would be a necessary ingredient for more advanced tasks. 

During training, decoder also uses information about the true output. It is feeded in as input symbol by symbol. However, during the prediction stage (which is called *inference* in this architecture), the decoder can only use its own generated output from the previous step to feed it in at the next step. Because of this difference (*training* vs *inference*), we will create two distinct instances, which will serve for the described scenarios.

The picture below illustrates the point. It also shows our work with the special characters, e.g. look how the start symbol `^` is used. The transparent parts are ignored. In decoder, it is masked out in the loss computation. In encoder, the green state is considered as final and passed to the decoder. 

<img src="encoder-decoder-pic.png" style="width: 500px;">

In [19]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [20]:
import tensorflow.keras.layers as L

In [21]:
import tensorflow_addons as tfa

Setting some hyperparameters:

In [22]:
vocab_size=15
embeddings_size=20
max_iter=7
hidden_size=512
start_id=1
end_id=2
padding_symbol_id =0
dropout=0.5
learning_rate = 0.001

Let's define encoder and decoder below:

In [33]:
class EncoderNet(tf.keras.Model):
    
    def __init__(self, name='EncoderNet'):
        
        super().__init__(name=name)
        
        self.embedding = L.Embedding(vocab_size,
                                     embeddings_size,
                                    ) 
        self.rnn = L.GRU(hidden_size, dropout=dropout)
                        

In [34]:
encoder = EncoderNet()

Now, it's time to implement the decoder using BasicDecoder interface in TensorFLow. We also need a sampler for trainig and another for inference.

As before, we should choose some RNN cell, e.g. GRU cell. To turn hidden states into logits, we will need a projection layer (dense without bias). 

In [35]:
class DecoderNet(tf.keras.Model):
    
    def __init__(self):
        
        super().__init__()
        
        self.embedding = L.Embedding(vocab_size,embeddings_size)
        self.dense = L.Dense(vocab_size)
        self.grucell = L.GRUCell(hidden_size, dropout=dropout)
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        # an instance of BaseDecoder
        self.decoder = tfa.seq2seq.BasicDecoder(self.grucell, 
                                        sampler= self.sampler,
                                        output_layer=self.dense,
                                               )
    

In [36]:
decoder = DecoderNet()

In [37]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


In this task we will use [sequence_loss](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/sequence_loss), which is a weighted cross-entropy loss for a sequence of logits. Also note, that we do not want to take into account loss terms coming from padding symbols, so we will mask them out using weights.  

In [38]:
def loss_function(y_pred, y, lengths):
   
    weights = tf.cast(tf.sequence_mask(lengths), dtype=tf.float32)
    loss = tfa.seq2seq.sequence_loss(y_pred, y, weights)
    return loss


def train_step(input_batch, ground_truth, ground_truth_lengths):
    
    with tf.GradientTape() as tape:
        input_batch_embed = encoder.embedding(input_batch)
        final_state = encoder.rnn(input_batch_embed)

        
        batch_size = tf.shape(input_batch)[0]
        start_tokens = tf.fill([batch_size], start_id)
        ground_truth_as_input = tf.concat([tf.expand_dims(start_tokens, 1), ground_truth], 1)

        
        ground_truth_embedded = decoder.embedding(ground_truth_as_input) 

                
        #BasicDecoderOutput 
        outputs, _, _  = decoder.decoder(ground_truth_embedded,
                                        initial_state=[final_state],
                                        )
        
        logits = outputs.rnn_output
        
        # dont predict the last token
        loss = loss_function(logits[:,:-1,:], ground_truth, ground_truth_lengths)
        
        

    #Returns the list of all layer variables / weights.
    variables = encoder.trainable_variables + decoder.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss


The following produces predictions by the trained model:

In [39]:
def predictions(X_batch):
    
    X_batch, _ = batch_to_ids(X_batch, word2id, max_len)

    X = tf.convert_to_tensor(X_batch, dtype=tf.int32)

    encoder_initial_cell_state = [tf.zeros((batch_size, hidden_size))]

    input_batch_embed = encoder.embedding(X)
    final_state = encoder.rnn(input_batch_embed,
                             initial_state =encoder_initial_cell_state)


    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

    start_tokens = tf.fill([batch_size], start_id)
    decoder_input = tf.expand_dims(start_tokens, 1)

    decoder_emb_inp = decoder.embedding(decoder_input)

    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.grucell, sampler = greedy_sampler,
                                            output_layer=decoder.dense)

    #initialize inference decoder
    decoder_embedding_matrix = decoder.embedding.variables[0] 
    (first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_id,
                             initial_state = [final_state])

    inputs = first_inputs
    state = first_state  
    predictions = np.empty((batch_size,0), dtype = np.int32)                                                                             
    for j in range(max_len*2):
        outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
        inputs = next_inputs
        state = next_state
        outputs = np.expand_dims(outputs.sample_id,axis = -1)
        predictions = np.append(predictions, outputs, axis = -1)

    pred = []
    for i in range(len(predictions)):
        line = predictions[i,:]
        seq = list(itertools.takewhile(lambda index: index !=2, line))
        pred.append("".join([id2word[w] for w in seq]))

    return pred


In [40]:
batch_size = 128 
max_len =20 

n_step = int(len(train_set) / batch_size)

## Train the network and prediction

In [41]:
invalid_number_prediction_counts = []
all_model_predictions = []
all_ground_truth = []


n_epochs = 15 



print('Start training... \n')
for epoch in range(n_epochs):  
    np.random.shuffle(train_set)
    np.random.shuffle(test_set)
    
    loss = 0

    print('Train: epoch', epoch + 1)
    for n_iter, (X_batch, Y_batch) in enumerate(generate_batches(train_set, batch_size=batch_size)):
        
        
        X,X_seq_len = batch_to_ids(X_batch, word2id, max_len)
        Y,Y_seq_len = batch_to_ids(Y_batch, word2id, max_len)
                
        X = tf.convert_to_tensor(X, dtype=tf.int32)
        Y = tf.convert_to_tensor(Y, dtype=tf.int32)
        Y_seq_len = tf.convert_to_tensor(Y_seq_len, dtype=tf.int32)
        
        batch_loss = tf.keras.backend.eval(train_step(X, Y, Y_seq_len))
        loss += batch_loss
        
        mae = []
        
        if n_iter % 200 == 0:
            X_test_batch, Y_test_batch = next(generate_batches(test_set, batch_size=batch_size))
            if len(X_test_batch) == batch_size:
                pred = predictions(X_test_batch)
                pred = [int(i) for i in pred]
                true = [int(i) for i in Y_test_batch]
                mae.append(mean_absolute_error(true, pred))


        
            print("Epoch: [%d/%d], step: [%d/%d], loss: %f, MAE for test: %f" % (epoch + 1, n_epochs, n_iter + 1, 
                                                               n_step, batch_loss,sum(mae)/len(mae)
                                                              ))
            

Start training... 

Train: epoch 1
Epoch: [1/15], step: [1/625], loss: 2.708197, MAE for test: 7021.468750
Epoch: [1/15], step: [201/625], loss: 1.870707, MAE for test: 4282.562500
Epoch: [1/15], step: [401/625], loss: 1.782158, MAE for test: 3346.390625
Epoch: [1/15], step: [601/625], loss: 1.634863, MAE for test: 1245.242188
Train: epoch 2
Epoch: [2/15], step: [1/625], loss: 1.599263, MAE for test: 1255.328125
Epoch: [2/15], step: [201/625], loss: 1.482839, MAE for test: 540.695312
Epoch: [2/15], step: [401/625], loss: 1.409111, MAE for test: 425.656250
Epoch: [2/15], step: [601/625], loss: 1.364343, MAE for test: 333.226562
Train: epoch 3
Epoch: [3/15], step: [1/625], loss: 1.364885, MAE for test: 441.750000
Epoch: [3/15], step: [201/625], loss: 1.316227, MAE for test: 333.515625
Epoch: [3/15], step: [401/625], loss: 1.285151, MAE for test: 236.390625
Epoch: [3/15], step: [601/625], loss: 1.260910, MAE for test: 182.859375
Train: epoch 4
Epoch: [4/15], step: [1/625], loss: 1.309271,